In [1]:
import tensorflow as tf 
import matplotlib 
import matplotlib.pyplot as plt
import numpy as np 
import sklearn
from sklearn.metrics import confusion_matrix
import pandas as pd
import seaborn as sn
import sys


In [2]:
! pip install -q kaggle
from google.colab import files

files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list #ghir pour tester the kaggle datasets
!kaggle datasets download trolukovich/food11-image-dataset/download -p /content/sample_data/ --unzip

Saving kaggle.json to kaggle.json
ref                                                       title                                               size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
ruchi798/data-science-job-salaries                        Data Science Job Salaries                            7KB  2022-06-15 08:59:12           6185        208  1.0              
surajjha101/bigbasket-entire-product-list-28k-datapoints  BigBasket Entire Product List (~28K datapoints)      6MB  2022-06-22 12:51:18           1950         87  1.0              
victorsoeiro/netflix-tv-shows-and-movies                  Netflix TV Shows and Movies                          2MB  2022-05-15 00:01:23          16605        483  1.0              
sameepvani/nasa-nearest-earth-objects                     NAS

In [3]:
from os import listdir
onlyfiles = [f for f in listdir("/content/sample_data/training")]
print(onlyfiles)

['Dairy product', 'Bread', 'Seafood', 'Egg', 'Meat', 'Noodles-Pasta', 'Dessert', 'Vegetable-Fruit', 'Fried food', 'Rice', 'Soup']


In [4]:
# Load Data
from os import listdir
onlyfiles = [f for f in listdir("/content/sample_data/training")]
class_names = onlyfiles

train_dir = "/content/sample_data/training"
val_dir   = "/content/sample_data/validation"
test_dir  = "/content/sample_data/evaluation"

target_size = (224,224)
batch_size = 16


train_ds = tf.keras.preprocessing.image_dataset_from_directory(
                train_dir,
                class_names = class_names,
                batch_size = batch_size,
             #   labels = 'inferred',
                label_mode = 'categorical',
                image_size = target_size,
                shuffle = True,
)

valid_ds = tf.keras.preprocessing.image_dataset_from_directory(
                val_dir,
                class_names = class_names,
                batch_size = batch_size,
               # labels = 'inferred',
                label_mode = 'categorical',
                image_size = target_size,
                shuffle = True,
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
                test_dir,
                class_names = class_names,
                batch_size = batch_size,
              #  labels = 'inferred',
                label_mode = 'categorical',
                image_size = target_size,
                shuffle = False,
)

# buffered prefetching
train_ds = train_ds.prefetch(buffer_size=16)
valid_ds = valid_ds.prefetch(buffer_size=16)

Found 9866 files belonging to 11 classes.
Found 3430 files belonging to 11 classes.
Found 3347 files belonging to 11 classes.


In [5]:
print(test_ds)

<BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 11), dtype=tf.float32, name=None))>


In [6]:
# Definir le modèle
model = tf.keras.Sequential()
model.add(tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(224,224, 3)))
model.add(tf.keras.applications.InceptionResNetV2(input_shape=(224,224, 3), include_top=False))
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.Dense(11, activation='softmax'))

print(model.summary()) 
#tf.keras.utils.plot_model(model, show_shapes=True) # graph du modèle

219070464/219055592 [==============================] - 2s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 inception_resnet_v2 (Functi  (None, 5, 5, 1536)       54336736  
 onal)                                                           
                                                                 
 global_average_pooling2d (G  (None, 1536)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 11)                16907     
                                                                 
Total params: 54,353,643
Trainable params: 54,293,099
Non-trainable params: 60,544
______________________________________

In [ ]:
# Train

model.compile(
                  loss = "categorical_crossentropy",
              optimizer='adam',
                  metrics = ['accuracy', 'AUC']) # AUC = Area under the ROC Curve
    
history = model.fit(train_ds, epochs = 15, validation_data = valid_ds) # Train and validate

#Je dois add 20 epochs, loss still too high

Epoch 1/15
617/617 [==============================] - 261s 372ms/step - loss: 1.0367 - accuracy: 0.6544 - auc: 0.9433 - val_loss: 1.7226 - val_accuracy: 0.5822 - val_auc: 0.9032
Epoch 2/15
617/617 [==============================] - 228s 369ms/step - loss: 0.6606 - accuracy: 0.7863 - auc: 0.9751 - val_loss: 1.0354 - val_accuracy: 0.6936 - val_auc: 0.9461
Epoch 3/15
617/617 [==============================] - 228s 369ms/step - loss: 0.5082 - accuracy: 0.8277 - auc: 0.9848 - val_loss: 1.0927 - val_accuracy: 0.7009 - val_auc: 0.9439
Epoch 4/15
617/617 [==============================] - 228s 370ms/step - loss: 0.3859 - accuracy: 0.8723 - auc: 0.9900 - val_loss: 2.1541 - val_accuracy: 0.5673 - val_auc: 0.8769
Epoch 5/15
487/617 [======================>.......] - ETA: 43s - loss: 0.3204 - accuracy: 0.8916 - auc: 0.9929

In [ ]:
# Test
model.evaluate(test_ds)

In [ ]:
print(labels)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=target_size,
    batch_size=batch_size,
    color_mode='rgb',
    shuffle=False,    
    class_mode='categorical')

In [ ]:
#Confusion Matrix
#preds=model.predict(test_ds)
predsTwo=model.predict(test_generator)
y_pred = np.argmax(predsTwo,axis=1)

y_true =[]
for images, labels in test_ds.unbatch().take(3347):
  y_true.append(labels.numpy())
y_true = np.array(y_true)

y_true = np.argmax(y_true)


cm = confusion_matrix(y_true, y_pred)
print('Confusion Matrix\n', cm)




In [ ]:
# Generate Confusion Matrix

y_pred = model.predict(test_ds, batch_size=32)
    
y_true = []
for images, labels in test_ds.unbatch().take(3347):
    y_true.append(labels.numpy())
y_true = np.array(y_true)

# Convert one-hot encoding to integers
y_true = np.argmax(y_true, axis=1)
y_pred = np.argmax(y_pred, axis=1)

cm = confusion_matrix(y_true, y_pred) # Generate Confusion Matrix
print(cm)
#df_cm = pd.DataFrame(cm, index = [i for i in class_names], columns = [i for i in class_names])
#matplotlib.pyplot.figure(figsize = (10,8))
#sn.heatmap(df_cm, annot=True)
#matplotlib.pyplot.savefig('./confusion_matrix')
'''
class_dist = []
for i in range(11):
    class_dist.append( np.sum(y_true == i))
print("True distribution of classes are:")

print(np.around(class_dist / np.sum(class_dist) * 100, 2)) # True distribution of classes
'''


In [ ]:
import numpy as np
pip install scikit-plot
import scikitplot as skplt
from sklearn.metrics import classification_report, confusion_matrix, roc_curve

preds=model.predict(test_generator)
y_pred = np.argmax(preds,axis=1)

y_true = test_generator.classes
print(classification_report(y_true, y_pred, target_names=labels))



#
roc = roc_curve(y_true, y_pred)
skplt.metrics.plot_roc(y_true, y_pred, pos_label=1)
plt.show()


#
scikitplot.metrics.plot_lift_curve(y_true, y_pred, title='Lift Curve')
plt.show()

In [ ]:
#Je peux essayer aussi 
'''
pip install scikit-plot

from sklearn.metrics import roc_auc_score, auc, roc_curve
preds = model.predict(test_generator)

y_pred = np.argmax(preds,axis=1)
y_true = test_generator.classes
fpr, tpr,  _  = roc_curve(y_true, y_pred, pos_label=1)
#roc_auc_score(y_true, y_pred)
#print (auc(fprate, tprate))
#print ('ROC Score:', roc_auc_score(y_true, y_pred, pos_label=1))
plt.plot(fpr,tpr)
plt.show()
'''

#or 
'''
import scikitplot as skplt
from sklearn.metrics import roc_curve
from matplotlib import pyplot as plt
preds = model.predict(test_generator)

y_pred = np.argmax(preds,axis=1)
y_true = test_generator.classes
skplt.metrics.plot_roc(y_true, y_pred, pos_label=1)
plt.show()
'''

#or

'''
from sklearn.metrics import roc_curve
preds=model.predict(test_generator)
y_pred = np.argmax(preds,axis=1)
y_true = test_generator.classes
roc = roc_curve(y_true, y_pred)
'''

In [ ]:
img = keras.preprocessing.image.load_img(
    "/content/sample_data/training/Fried food/209.jpg", target_size=(224, 224)
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create batch axis

predictions = model.predict(img_array)
score = predictions[0][0]
print(
    "This image is %.2f percent dessert and %.2f percent not dessert."
    % (100 * (1 - score), 100 * score)
)
print(predictions)

In [ ]:
# Metrics plots
matplotlib.pyplot.figure()
matplotlib.pyplot.plot(history.history['loss'])
matplotlib.pyplot.savefig('./loss')
matplotlib.pyplot.figure()
matplotlib.pyplot.plot(history.history['accuracy'])
matplotlib.pyplot.savefig('./accuracy')
matplotlib.pyplot.figure()
matplotlib.pyplot.plot(history.history['auc'])
matplotlib.pyplot.savefig('./auc')
matplotlib.pyplot.figure()
matplotlib.pyplot.plot(history.history['val_loss'])
matplotlib.pyplot.savefig('./val_loss')
matplotlib.pyplot.figure()
matplotlib.pyplot.plot(history.history['val_accuracy'])
matplotlib.pyplot.savefig('./val_accuracy')
matplotlib.pyplot.figure()
matplotlib.pyplot.plot(history.history['val_auc'])
matplotlib.pyplot.savefig('./val_auc')

In [ ]:
# Save model and weights
model.save('./model')
model.save_weights('./weights')
!zip -r -q Adamax4_plots.zip ./confusion_matrix.png ./loss.png ./accuracy ./auc.png ./val_loss.png ./val_accuracy.png ./val_auc.png # Zip all plots
!zip -r -q Adamax4.zip ./model ./confusion_matrix.png ./weights.data-00000-of-00001 ./weights.index ./loss ./accuracy ./auc ./val_loss ./val_accuracy ./val_auc